In [39]:
from transformers import AlbertForMaskedLM, AutoTokenizer
import torch

In [70]:
model = AlbertForMaskedLM.from_pretrained(r"D:\CODE\Python\Transformers-Models\albert-base-v2")

In [71]:
tokenizer = AutoTokenizer.from_pretrained(r"D:\CODE\Python\Transformers-Models\albert-base-v2")

In [85]:
origin = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint."
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."
sequence


'Distilled models are smaller than the models they mimic. Using them instead of the large versions would help [MASK] our carbon footprint.'

In [86]:
input_ids = tokenizer.encode(sequence, return_tensors="pt")
label_ids = tokenizer.encode(origin, return_tensors="pt")

In [87]:
input_ids != label_ids

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False]])

In [88]:
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
mask_token_index

tensor([22])

In [89]:
output = model(input_ids, labels=label_ids)

In [90]:
output.keys()

odict_keys(['loss', 'logits'])

In [92]:
# reduce 1.2697  increace 1.4204
output.loss

tensor(1.4204, grad_fn=<NllLossBackward>)

In [79]:
token_logits = output.logits
token_logits.shape

torch.Size([1, 28, 30000])

In [80]:
mask_token_logits = token_logits[0, mask_token_index, :]

In [81]:
mask_token_logits

tensor([[ -3.6841,   3.2623,  -5.8545,  ..., -10.5368,  -3.2228,  -5.2283]],
       grad_fn=<IndexBackward>)

In [93]:
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

In [101]:
sequence = torch.topk(token_logits[0], 1, dim=-1).indices[0].tolist()
# sequence

In [83]:
tokenizer.decode(top_5_tokens)

'reduce minimize decrease increase offset'

In [102]:
tokenizer.decode(sequence)

'time'